In [260]:
#%% ---------------------------------Libraries---------------------------------
from __future__ import division
from __future__ import print_function
# import torch
import pdb
import glob
import sys
import random
import numpy as np
import matplotlib.pyplot as plt

import time
import os
import copy as cp
from collections import deque
import pandas as pd
from scipy.optimize import curve_fit

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

import lightgbm as lgbm

import keras
from keras.models import Sequential
from keras.layers import *

from tqdm import tqdm_notebook as tqdm
%matplotlib inline

In [261]:
glob.glob('data/*.csv')

['data/best_features_scores.csv',
 'data/hotelbedRooms.csv',
 'data/geohash_3_pairs.csv',
 'data/best_features_scores_3.csv',
 'data/OldData_Unmatched_pairs.csv',
 'data/normalized_32_feature_balanced_200k.csv',
 'data/OldData_matched_pairs.csv',
 'data/new_32_feature_balanced_200k.csv',
 'data/dotwRooms.csv',
 'data/best_features_scores_1.csv',
 'data/geohash_normalized_test_32_feature.csv',
 'data/best_features_scores_all.csv',
 'data/matched_pairs.csv',
 'data/geohash_test_32_feature.csv',
 'data/best_features_scores_2.csv',
 'data/Unmatched_pairs.csv']

In [17]:
df = pd.read_csv("data/normalized_32_feature_balanced_200k.csv") 
y = df.iloc[:, 0].values
X = df.iloc[:, 1:].values

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [299]:
def init_pop(FILE_PATH, features_col = 1):
    '''
    This method if you want to initialize the population using predefined values in csv file
    Hint: you have to pass the return value to populate method.
    '''
    df = pd.read_csv(FILE_PATH)
    init_arr = []
    for seq in df.iloc[:,features_col]:
        init_arr.append(eval(seq))
    init_arr = np.array(init_arr)
    return init_arr

In [296]:
def populate(POP_SIZE=20, IND_SIZE=5, init_arr = 'NONE'):
    if init_arr == 'NONE':
        pop = np.random.randint(0,2,(POP_SIZE, IND_SIZE), dtype= np.bool) *1
    else:
        pop1 = np.array(init_arr, dtype= np.bool)*1
        POP_SIZE = POP_SIZE - pop1.shape[0]
        IND_SIZE = pop1.shape[1]
        pop2 = np.random.randint(0,2,(POP_SIZE, IND_SIZE), dtype= np.bool) *1
        pop = np.concatenate((pop1,pop2))
    return pop

In [292]:
pop = populate()
pop

array([[0, 1, 0, 1, 0],
       [0, 0, 0, 0, 0],
       [1, 1, 1, 0, 0],
       [1, 1, 1, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 1, 1, 0],
       [0, 0, 1, 1, 1],
       [1, 1, 1, 1, 0],
       [0, 1, 1, 0, 1],
       [1, 1, 1, 0, 1],
       [0, 1, 0, 1, 1],
       [1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 1, 1, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 0, 1, 1],
       [1, 0, 0, 1, 1],
       [1, 0, 1, 0, 1]])

In [48]:
def objf_deep(X_train, X_test, y_train, y_test):
    # Define the model
    RUN_NAME = "model1"
    model = Sequential(name=RUN_NAME)
    input_dim = X_train.shape[1]
    model.add(Dense(5, input_dim=input_dim, activation='relu',
                kernel_initializer='he_uniform', name='layer_1'))
    model.add(Dense(10, activation='relu', name='layer_2'))
    model.add(Dropout(0.1))
    model.add(Dense(5, activation='relu', name='layer_3'))
    #     model.add(Dropout(0.1))
    model.add(Dense(1, activation='sigmoid', name='output_layer'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

    es = keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=5)
    # Train the model
    hist = model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    batch_size = 100,
    epochs=5,
    shuffle=True,
    verbose=0,
    callbacks=[es]
    )

    y_pred=model.predict_classes(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    loss = 1- accuracy
    return loss

In [156]:
def objf_lgbm(X_train, X_test, y_train, y_test):
    lgbm_clf = lgbm.LGBMClassifier()
    lgbm_clf.fit(X_train, y_train)
    y_pred=lgbm_clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    loss = 1- accuracy
    return loss

In [222]:
idx = np.where(pop[1] == 1)[0].tolist()
new_X_train = X_train[:, idx]
new_X_test = X_test[:, idx]
objf_lgbm(new_X_train, new_X_test, y_train, y_test)

0.07936507936507942

In [88]:
def mutate(indv, mutate_rate):
    '''
    this function return the mutation for chiled by mutation rate as a factor.
    '''
    mutate_value = int(round(mutate_rate * indv.shape[0]))
    mut_indx = np.random.randint(0, indv.shape[0], mutate_value).tolist()
    indv[mut_indx] = 1 - indv[mut_indx]
    return indv

In [90]:
indv = pop[1]
indv

array([1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 0, 1, 0, 1, 1, 0, 1, 1, 1])

In [91]:
mutate_rate = 0.1
mutate(indv, mutate_rate)

array([1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 0, 1])

In [210]:
def crossover2(indv1, indv2, crossover_rate):
    '''
    this function return the cross over between two agents defined by crossover rate.
    '''
    indv1, indv2 = np.array(indv1), np.array(indv2)
    crossover_value = int(round(crossover_rate * len(indv1),0))
    
    flag = np.random.randint(0,2)
    if flag==1:
        child = np.hstack((indv1[:crossover_value] , indv2[crossover_value:]))
        
    else:
        child = np.hstack((indv2[:crossover_value] , indv1[crossover_value:]))
    return child

In [211]:
indv1

array([1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 0, 1])

In [229]:
indv1 = pop[1]
indv2 = pop[2]
crossover_rate = 0.5
crossover2(indv1, indv2, crossover_rate)

array([0, 0, 0, 1, 1, 1, 1, 0, 1, 0])

In [238]:
def crossover3(indv1, indv2, indv3, crossover_rate):
    '''
    this function return the cross over between three indvs defined by crossover rate.
    '''
    indv1, indv2, indv3 = np.array(indv1), np.array(indv2), np.array(indv3)
    crossover_value = int(round(crossover_rate * len(indv1),0))
    flag = np.random.randint(0,3)
    if flag == 0: 
        child = np.hstack((indv1[:crossover_value] , 
                           indv2[crossover_value:crossover_value*2], 
                           indv3[crossover_value*2:]))
    elif flag == 1: 
        child = np.hstack((indv1[:crossover_value] , 
                           indv3[crossover_value:crossover_value*2], 
                           indv2[crossover_value*2:]))
    else: 
        child = np.hstack((indv3[:crossover_value] , 
                           indv2[crossover_value:crossover_value*2], 
                           indv1[crossover_value*2:]))
    return child

In [241]:
indv1 = pop[1]
indv2 = pop[2]
indv3 = pop[3]
crossover_rate = 0.3
crossover3(indv1, indv2, indv3, crossover_rate)

array([0, 0, 0, 1, 0, 1, 0, 0, 0, 0])

In [300]:
gen_no=1*20; run_no = 5;POP_SIZE = 25;mutate_rate = 0.2

In [301]:
columns = ['run_no','loss','Excec_Time','best_solution', 'best_10_solutions']
df = pd.DataFrame(columns = columns)
conv_df = pd.DataFrame(np.zeros((gen_no)))

for run_no_i in range(run_no):

    Convergence_curve = []                    
                       
    Last_Alpha_score =float("inf")

    IND_SIZE = X_train.shape[1]
    #-------------------------------------------------

#     pdb.set_trace()
    FILE_PATH = 'data/best_features_scores_all.csv'
    init_arr = init_pop(FILE_PATH)
    Positions = populate(POP_SIZE, init_arr=init_arr)

    pos_fit = []

    fitness_arr = np.array([[1, np.asarray(Positions[0])]])
    #--------------------------------------------------
    timerStart = time.time()
    df_list = []
    #--------------------------------------------------    
    for gen_no_i in tqdm(range(gen_no)):
#         sys.stdout.write("\r%.2f%%" % ((gen_no_i+1)*100/gen_no))
#         sys.stdout.flush()
        for agent in  Positions:
            idx = np.where(agent == 1)[0].tolist()
            new_X_train = X_train[:, idx]
            new_X_test = X_test[:, idx]
            fitness = objf_lgbm(new_X_train, new_X_test, y_train, y_test)
            arr1 = np.array([[fitness, np.asarray(agent)]])
            fitness_arr=np.vstack((fitness_arr,arr1))  
            pos_fit.append([fitness, agent])

        fitness_arr = fitness_arr[fitness_arr[:,0].argsort()]    
        Alpha_score, Alpha_pos = fitness_arr[0][0], list(fitness_arr[0][1])
        Beta_score, Beta_pos   = fitness_arr[1][0], list(fitness_arr[1][1])
        Delta_score, Delta_pos = fitness_arr[2][0], list(fitness_arr[2][1])
    #---------------------------------------------------------------------    
        if Alpha_score<Last_Alpha_score:
            Last_Alpha_score = Alpha_score
        Convergence_curve.append(Last_Alpha_score)
        
    #--------------------------Start prey search----------------------------------------------------    
    #       a=2-l*((2)/gen_no); # a decreases linearly fron 2 to 0 --> a =
        a= 2*(1-(gen_no_i/gen_no)**2)
        for i in range(POP_SIZE):               
            X1,X2,X3 = [], [], []
            r1=random.random() # if r1>0.5: A1=[0:2] -->converge --> crossover
            r2=random.random() # if r1<0.5: A1=[-2:0] --> Diverge --> Mutation
            A1=2*a*r1-a; # Equation (3.3)
            C1=2*r2; # Equation (3.4)
            B0 = []            
            agent_fitness = pos_fit[i][0]
            B0 = cp.deepcopy(pos_fit[i][1])
            D_alpha = abs((agent_fitness - C1*Alpha_score)/agent_fitness) 
            if abs(A1)>=1:
                #Exploration
                X1 = mutate(B0, D_alpha)
            if abs(A1)<1:
                #Exploitation
                X1 = crossover2(Alpha_pos, B0, D_alpha)

            r1=random.random() # if r1>0.5: A1=[0:2] -->converge --> crossover
            r2=random.random() # if r1<0.5: A1=[-2:0] --> Diverge --> Mutation
            A2=2*a*r1-a; # Equation (3.3)
            C2=2*r2; # Equation (3.4)

            B0 = []
            B0 = cp.deepcopy(pos_fit[i][1])
            D_beta = abs((agent_fitness - C2*Beta_score)/agent_fitness)
            if abs(A2)>=1:
                #Exploration
                X2 = mutate(B0, D_beta)
            if abs(A2)<1:
                #Exploitation
                X2 = crossover2(Beta_pos, B0, D_beta)

            r1=random.random() # if r1>0.5: A1=[0:2] -->converge --> crossover
            r2=random.random() # if r1<0.5: A1=[-2:0] --> Diverge --> Mutation
            A3=2*a*r1-a; # Equation (3.3)
            C3=2*r2; # Equation (3.4)

            B0 = []
            B0 = cp.deepcopy(pos_fit[i][1])
            D_delta = abs((agent_fitness - C3*Delta_score)/agent_fitness)
            if abs(A3)>=1:
                #Exploration
                X3 = mutate(B0, D_delta)
            if abs(A3)<1:
                #Exploitation
                X3 = crossover2(Delta_pos, B0, D_delta)

            Positions[i] = crossover3(X1, X2, X3, 0.3)

        pos_fit = []   
        # Remove All Rows from fitness_arr Except first three (Alpha_ps, Beta_Pos, Delta_pos)
        fitness_arr = np.delete(fitness_arr,np.s_[3:], axis=0)

    Positions = np.array(pd.DataFrame(Positions).drop_duplicates())
    fitness_arr = np.zeros((len(Positions),2), dtype='object')
    for i,agent in  enumerate(Positions):
        idx = np.where(agent == 1)[0].tolist()
        new_X_train = X_train[:, idx]
        new_X_test = X_test[:, idx]
        fitness = objf_lgbm(new_X_train, new_X_test, y_train, y_test)
        fitness_arr[i,0] = fitness
        fitness_arr[i,1] = agent.tolist()

    fitness_arr = fitness_arr[fitness_arr[:,0].argsort()] 
    df_list = fitness_arr.tolist()
    df = pd.DataFrame(df_list, columns=['loss', 'features'])
    df.to_csv('data/best_features_scores_{}.csv'.format(run_no), index=False)

/home/ai/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  


/home/ai/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  


/home/ai/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  


/home/ai/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  


/home/ai/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  


In [303]:
agent = np.array([0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1])

In [304]:
idx = np.where(agent == 1)[0].tolist()
new_X_train = X_train[:, idx]
new_X_test = X_test[:, idx]
fitness = objf_lgbm(new_X_train, new_X_test, y_train, y_test)

In [305]:
fitness

0.048688094076088206

In [306]:
idx

[2, 6, 12, 13, 15, 16, 17, 19, 21, 23, 24, 26, 28, 29, 30, 31]

In [309]:
SelectedFeatureInds = [0, 3, 4, 5, 6, 7, 8, 9, 10, 11, 21, 22, 23, 24, 29, 30, 31]
idx=np.zeros(X.shape[1], dtype='bool')
idx[SelectedFeatureInds]=1
idx = idx.tolist()
new_X_train = X_train[:, idx]
new_X_test = X_test[:, idx]
fitness = objf_lgbm(new_X_train, new_X_test, y_train, y_test)

In [310]:
fitness

0.021404169281182495

In [311]:
new_X_train = X_train
new_X_test = X_test
fitness = objf_lgbm(new_X_train, new_X_test, y_train, y_test)
fitness

0.01949847777080571